In [23]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras import datasets, applications

In [24]:
#use this string for the path to the folder containing all the images in kaggle

folder_path = '/kaggle/input/ucf101/'

class_indices_file = open(folder_path+ 'UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/classInd.txt', 'r')
class_index = {}
for line in class_indices_file:
    class_ = line.split(' ')
    class_index[class_[1].strip('\n')] = int(class_[0])

class_indices_file.close()

print(class_index)

{'ApplyEyeMakeup': 1, 'ApplyLipstick': 2, 'Archery': 3, 'BabyCrawling': 4, 'BalanceBeam': 5, 'BandMarching': 6, 'BaseballPitch': 7, 'Basketball': 8, 'BasketballDunk': 9, 'BenchPress': 10, 'Biking': 11, 'Billiards': 12, 'BlowDryHair': 13, 'BlowingCandles': 14, 'BodyWeightSquats': 15, 'Bowling': 16, 'BoxingPunchingBag': 17, 'BoxingSpeedBag': 18, 'BreastStroke': 19, 'BrushingTeeth': 20, 'CleanAndJerk': 21, 'CliffDiving': 22, 'CricketBowling': 23, 'CricketShot': 24, 'CuttingInKitchen': 25, 'Diving': 26, 'Drumming': 27, 'Fencing': 28, 'FieldHockeyPenalty': 29, 'FloorGymnastics': 30, 'FrisbeeCatch': 31, 'FrontCrawl': 32, 'GolfSwing': 33, 'Haircut': 34, 'Hammering': 35, 'HammerThrow': 36, 'HandstandPushups': 37, 'HandstandWalking': 38, 'HeadMassage': 39, 'HighJump': 40, 'HorseRace': 41, 'HorseRiding': 42, 'HulaHoop': 43, 'IceDancing': 44, 'JavelinThrow': 45, 'JugglingBalls': 46, 'JumpingJack': 47, 'JumpRope': 48, 'Kayaking': 49, 'Knitting': 50, 'LongJump': 51, 'Lunges': 52, 'MilitaryParade': 

In [25]:
req_classes = [[1,2], [8,9], [23,24], [30], [87], [70], [72], [100], [59,60,61,62,63,64,65,66,67], 
               [83],[98], [29], [85], [79], [80], [84], [51], [45], [77], [73], [44]]

class_names = []
for _ in req_classes:
    classes = []
    for __ in _:
      classes.append([key for key in list(class_index.keys()) if class_index[key] == __][0])
    class_names.append(classes)
print(class_names)


[['ApplyEyeMakeup', 'ApplyLipstick'], ['Basketball', 'BasketballDunk'], ['CricketBowling', 'CricketShot'], ['FloorGymnastics'], ['SumoWrestling'], ['PullUps'], ['PushUps'], ['WritingOnBoard'], ['PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'PlayingViolin'], ['SkyDiving'], ['WalkingWithDog'], ['FieldHockeyPenalty'], ['SoccerPenalty'], ['Shotput'], ['SkateBoarding'], ['SoccerJuggling'], ['LongJump'], ['JavelinThrow'], ['SalsaSpin'], ['Rafting'], ['IceDancing']]


In [26]:
label_dict = {}

label = 1
for _ in req_classes:
    for id in _:
        label_dict[id] = label
    label += 1

print(label_dict)

{1: 1, 2: 1, 8: 2, 9: 2, 23: 3, 24: 3, 30: 4, 87: 5, 70: 6, 72: 7, 100: 8, 59: 9, 60: 9, 61: 9, 62: 9, 63: 9, 64: 9, 65: 9, 66: 9, 67: 9, 83: 10, 98: 11, 29: 12, 85: 13, 79: 14, 80: 15, 84: 16, 51: 17, 45: 18, 77: 19, 73: 20, 44: 21}


In [27]:
#considering only one test train split 01
train_path_file = open(folder_path + "UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist01.txt", "r")
train_path_list = []
train_y = []
# print(next(train_path_file))
list_req_class = [i for l in req_classes for i in l]
print(list_req_class)
for _ in train_path_file:
    path, class_num = _.split(' ')
    if (int(class_num.strip('\n')) in list_req_class):
        train_path_list.append(path)
        train_y.append(label_dict[int(class_num.strip('\n'))])
        #print(int(label_dict[class_num.strip('\n')]))
    
train_path_file.close()


[1, 2, 8, 9, 23, 24, 30, 87, 70, 72, 100, 59, 60, 61, 62, 63, 64, 65, 66, 67, 83, 98, 29, 85, 79, 80, 84, 51, 45, 77, 73, 44]


In [28]:
len(train_path_list)

3035

In [29]:
test_path_file = open(folder_path + "UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist01.txt", "r")
test_path_list = []
test_y = []
# print(next(test_path_file))
for _ in test_path_file:
    path = _.strip('\n')
    a,b = path.split('/')
    if (class_index[a] in list_req_class):
        test_path_list.append(path)
        test_y.append(label_dict[class_index[a]])


test_path_file.close()



In [30]:
len(test_path_list)

1214

In [31]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
train_y_hot = pd.get_dummies(train_y)
test_y_hot = pd.get_dummies(test_y)

In [32]:
labels = np.unique(np.array(train_y))

sig_frame_dict = {}
for i in labels:
    with open('/kaggle/input/train-hist-frames/'+ 'class_'+ str(i) +'_sig_frames.txt', 'r') as readfile:
        contents = readfile.read()
        lines = contents.splitlines()
    for l in lines:
        #print(l)
        l_arr = l.split(" ", 1)
        sig_frame_dict['/kaggle/input/ucf101/UCF101/UCF-101/'+ l_arr[0]] = eval(l_arr[1])

    with open('/kaggle/input/test-hist-frames/'+ 'test_class_'+ str(i) +'_sig_frames.txt', 'r') as read_testfile:
        contents = read_testfile.read()
        lines = contents.splitlines()
    for l in lines:
        l_arr = l.split(" ", 1)
        sig_frame_dict['/kaggle/input/ucf101/UCF101/UCF-101/' + l_arr[0]] = eval(l_arr[1])
    


In [33]:
sig_frame_dict['/kaggle/input/ucf101/UCF101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi']

[0, 1, 4, 6, 7, 12, 16, 22, 28, 33, 37, 42, 44, 48, 53, 58, 64, 72, 76, 84]

# ## **data loader**

## for skip interval/random/adaptive frame selection

In [34]:

class VideoGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, X=train_path_list, Y=train_y, batch_size=32, frame_selection = 'skip'):
        'Initialization'
        self.batch_size = batch_size
        self.X = X
        self.Y = Y
        self.frame_selection = frame_selection
       
        
    # X_paths, y_labels = generate_X_Y(self.train)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        
        
        if (index == len(self.X)//self.batch_size - 1 and (index+1)*self.batch_size > len(self.X)-1):
            indexes = np.arange(index*self.batch_size,len(self.X), 1)
        else:
            indexes = np.arange((index)*self.batch_size,(index+1)*self.batch_size,1)
        # Find list of paths
        list_paths = [self.X[k] for k in indexes]
        
        # Generate data
        X_batch = []
        y_batch =[]

      
        for ind in indexes:
            vid_path = self.X[ind]
            frame_arr = self.generate_frames_from_videos(folder_path+ 'UCF101/UCF-101/' + vid_path)
            X_batch.append(frame_arr)
            y_batch.append(self.Y[ind])
        
#         print(index, np.array(X_batch).shape)
        return applications.xception.preprocess_input(np.array(X_batch)), np.array(y_batch)

    def generate_frames_from_videos(self, video_path):

        #CAPTURING FRAMES after EVERY 20 FRAMES OF VIDEOS
  
        video1 = cv2.VideoCapture(video_path)
        #total_frames = video1.get(cv2.CAP_PROP_FRAME_COUNT)
        
        if (self.frame_selection == 'skip'):
            frames_total = []
            frames_arr = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)

            video1.release()
            total_frames = len(frames_total)
            #print(total_frames)
            interval = total_frames // 20

            for _ in range (total_frames):
                if _ % interval == 0:
                    resized_frame = cv2.resize(frames_total[_], (224, 224))
                    normalized_frame = resized_frame / 255
                    frames_arr.append(normalized_frame)

                if _ >= 19 * interval:
                    break

#             print(np.array(frames_arr).shape)
            return frames_arr
        
        
        elif (self.frame_selection == 'random'):
            
            frames_total = []
            frames_arr = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)

            video1.release()
            total_frames = len(frames_total)
            rand_list = np.random.randint(0,total_frames+1,20)

            for _ in rand_list:
                resized_frame = cv2.resize(frames_total[_], (224, 224))
                normalized_frame = resized_frame / 255
                frames_arr.append(normalized_frame)

            video1.release()
            return frames_arr
        
        
        elif (self.frame_selection == 'hist_difference'):
            frames_num = sig_frame_dict[video_path]
            #print(len(frames_num))
            frame_count = 0
            frames_arr = []
            
            frames_total = []
            while video1.isOpened():
                ret, frame = video1.read()

                if not ret:  #reached end of video
                    break
                frames_total.append(frame)
                
            total_frames = len(frames_total)
            
            actual = []
            
            for i in range(len(frames_num)):
                ind = list(frames_num)[i]
                if ind < total_frames:
                    actual.append(ind)
                else:
                    int_ind = np.random.randint(0,total_frames)
                    actual.append(int_ind)
            if (len(actual)< 20):
                actual = actual + list(np.random.randint(0,total_frames,20-len(actual)))
            for ind in actual:
                resized_frame = cv2.resize(frames_total[int(ind)], (224, 224))
                #normalized_frame = resized_frame / 255
                frames_arr.append(resized_frame)

            video1.release()
            #print(np.array(frames_arr).shape)
            return frames_arr
            


    

In [ ]:
def generate_frames_from_videos(video_path):
    frames_num = sig_frame_dict[video_path]
#     print(frames_num)
    #print(len(frames_num))
    frame_count = 0
    frames_arr = []
    video1 = cv2.VideoCapture(video_path)


    frames_total = []
    while video1.isOpened():
        ret, frame = video1.read()

        if not ret:  #reached end of video
            break
        frames_total.append(frame)
        
    video1.release()
    total_frames = len(frames_total)

    actual = []

    for i in range(len(frames_num)):
        ind = list(frames_num)[i]
        if ind < total_frames:
            actual.append(ind)
        else:
            int_ind = np.random.randint(0,total_frames)
            actual.append(int_ind)
    if (len(actual)< 20):
            actual = actual + list(np.random.randint(0,total_frames,20-len(actual)))
    for ind in actual:
        resized_frame = cv2.resize(frames_total[int(ind)], (224, 224))
        normalized_frame = resized_frame / 255
        frames_arr.append(normalized_frame)

    #print("frames shape:" , np.array(frames_arr).shape)
    return frames_arr
    
indices = np.arange(0,len(train_path_list),1)
list_paths = [train_path_list[k] for k in indices]
        
        # Generate data
X_batch = []
y_batch =[]


for ind in indices:
    vid_path = train_path_list[ind]
    frame_arr = generate_frames_from_videos(folder_path+ 'UCF101/UCF-101/' + vid_path)
    
    if(np.array(frame_arr).shape != (20, 224, 224, 3)):
        print(sig_frame_dict[folder_path+ 'UCF101/UCF-101/' + vid_path])
        print(vid_path, np.array(frame_arr).shape)

In [17]:
sig_frame_dict[folder_path+ 'UCF101/UCF-101/' + 'Basketball/v_Basketball_g09_c01.avi']

[0, 12, 24, 36, 37, 40, 43, 45, 47, 48, 56, 60, 65, 68, 72, 73, 75, 78, 80, 81]

## pretrained Xception 

In [13]:
Xcep_pt_model = applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    classes = len(req_classes)
)
Xcep_pt_model.trainable = False

83683744/83683744 [==============================] - 0s 0us/step


In [23]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(Xcep_pt_model, input_shape= (20,120,160,3)))
model_pret_x.add(TimeDistributed(Dropout(0.25)))

model_pret_x.add(TimeDistributed(Flatten()))

model_pret_x.add(LSTM(32))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))




In [24]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)


In [25]:
model_pret_x.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDis  (None, 20, 4, 5, 2048)   20861480  
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 20, 4, 5, 2048)   0         
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, 20, 40960)        0         
 tributed)                                                       
                                                                 
 lstm_1 (LSTM)               (None, 128)               21037568  
                                                                 
 dense_1 (Dense)             (None, 21)                2709      
                                                      

In [26]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [27]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 20, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 

Epoch 1/20
94/94 [==============================] - 306s 3s/step - loss: 3.2905 - accuracy: 0.1130 - val_loss: 2.6779 - val_accuracy: 0.3057
Epoch 2/20
94/94 [==============================] - 295s 3s/step - loss: 2.6081 - accuracy: 0.3135 - val_loss: 2.5052 - val_accuracy: 0.3083
Epoch 3/20
94/94 [==============================] - 293s 3s/step - loss: 2.5120 - accuracy: 0.3115 - val_loss: 2.4241 - val_accuracy: 0.3176
Epoch 4/20
94/94 [==============================] - 294s 3s/step - loss: 2.4001 - accuracy: 0.3348 - val_loss: 2.3536 - val_accuracy: 0.3784
Epoch 5/20
94/94 [==============================] - 295s 3s/step - loss: 2.3215 - accuracy: 0.3674 - val_loss: 2.2873 - val_accuracy: 0.3733
Epoch 6/20
94/94 [==============================] - 292s 3s/step - loss: 2.1953 - accuracy: 0.3933 - val_loss: 2.2229 - val_accuracy: 0.3936
Epoch 7/20
94/94 [==============================] - 293s 3s/step - loss: 2.0787 - accuracy: 0.4362 - val_loss: 2.1613 - val_accuracy: 0.4257
Epoch 8/20
94

In [ ]:
model_pret_x.save("xception.h5")

# inception v3

In [29]:
incep_pret_model = applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    classes = len(req_classes)
)
incep_pret_model.trainable = False

In [30]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(incep_pret_model, input_shape= (20,120,160,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))





In [31]:
model_pret_x.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_7 (TimeDis  (None, 20, 2, 3, 2048)   21802784  
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 20, 2048)         0         
 tributed)                                                       
                                                                 
 lstm_2 (LSTM)               (None, 128)               1114624   
                                                                 
 dense_2 (Dense)             (None, 21)                2709      
                                                                 
Total params: 22,920,117
Trainable params: 1,117,333
Non-trainable params: 21,802,784
_________________________________________________________________


In [32]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [37]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1
                                                                                           ), metrics = ["accuracy"])

### skip interval model

### lr = 0.1, momentum = 0

In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 

In [28]:
model_pret_x.save('/kaggle/working/model_incep_skip.h5')

### lr = 0.01, momentum= 0.1

In [40]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01,
                                momentum = 0.1), metrics = ["accuracy"])

In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'skip'),callbacks = [early_stopping_callback]) 

## **image size 224x224**

In [18]:
Xcep_pt_model = applications.xception.Xception(
    include_top=False,
    weights='imagenet',
)
Xcep_pt_model.trainable = False

83683744/83683744 [==============================] - 1s 0us/step


In [19]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(Xcep_pt_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))


model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))





In [20]:
model_pret_x.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 20, 7, 7, 2048)   20861480  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 20, 2048)         0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 128)               1114624   
                                                                 
 dense (Dense)               (None, 21)                2709      
                                                                 
Total params: 21,978,813
Trainable params: 1,117,333
Non-trainable params: 20,861,480
_________________________________________________________________


In [15]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)


In [23]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])


In [13]:
model_pret_x = keras.models.load_model('/kaggle/input/saved-models/model_xcep_224_hist_1.h5')

In [17]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 32, 
                            frame_selection = 'hist_difference'),epochs = 20, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 32, 
                            frame_selection = 'hist_difference'),callbacks = [early_stopping_callback]) 

Epoch 1/20
94/94 [==============================] - 435s 5s/step - loss: 1.8937 - accuracy: 0.4422 - val_loss: 1.8355 - val_accuracy: 0.4840
Epoch 2/20
94/94 [==============================] - 403s 4s/step - loss: 1.7394 - accuracy: 0.4914 - val_loss: 1.7362 - val_accuracy: 0.4949
Epoch 3/20
94/94 [==============================] - 392s 4s/step - loss: 1.6738 - accuracy: 0.5299 - val_loss: 1.6961 - val_accuracy: 0.4848
Epoch 4/20
94/94 [==============================] - 401s 4s/step - loss: 1.6436 - accuracy: 0.5296 - val_loss: 1.6496 - val_accuracy: 0.4823
Epoch 5/20
94/94 [==============================] - 392s 4s/step - loss: 1.5529 - accuracy: 0.5668 - val_loss: 1.5383 - val_accuracy: 0.5642
Epoch 6/20
94/94 [==============================] - 394s 4s/step - loss: 1.4511 - accuracy: 0.6273 - val_loss: 1.5039 - val_accuracy: 0.5752
Epoch 7/20
94/94 [==============================] - 396s 4s/step - loss: 1.3509 - accuracy: 0.6506 - val_loss: 1.3697 - val_accuracy: 0.6681
Epoch 8/20
94

In [18]:
model_pret_x.save('/kaggle/working/model_xcep_224_hist_2.h5')

## pretrained vgg 224 224 random

In [24]:
vgg_pret_model = applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
)
vgg_pret_model.trainable = False

In [34]:
vgg_pret_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [30]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(vgg_pret_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))






In [31]:
model_pret_x.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_5 (TimeDis  (None, 20, 7, 7, 512)    14714688  
 tributed)                                                       
                                                                 
 time_distributed_6 (TimeDis  (None, 20, 512)          0         
 tributed)                                                       
                                                                 
 lstm_2 (LSTM)               (None, 128)               328192    
                                                                 
 dense_2 (Dense)             (None, 21)                2709      
                                                                 
Total params: 15,045,589
Trainable params: 330,901
Non-trainable params: 14,714,688
_________________________________________________________________


In [32]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)



In [33]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])



In [36]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),callbacks = [early_stopping_callback]) 

Epoch 1/10
189/189 [==============================] - 402s 2s/step - loss: 2.7235 - accuracy: 0.2913 - val_loss: 2.4352 - val_accuracy: 0.3433
Epoch 2/10
189/189 [==============================] - 373s 2s/step - loss: 2.3501 - accuracy: 0.3310 - val_loss: 2.2935 - val_accuracy: 0.3250
Epoch 3/10
189/189 [==============================] - 396s 2s/step - loss: 2.1784 - accuracy: 0.4021 - val_loss: 2.0980 - val_accuracy: 0.4058
Epoch 4/10
189/189 [==============================] - 387s 2s/step - loss: 1.9907 - accuracy: 0.4454 - val_loss: 1.8713 - val_accuracy: 0.4700
Epoch 5/10
189/189 [==============================] - 375s 2s/step - loss: 1.8325 - accuracy: 0.4706 - val_loss: 1.7495 - val_accuracy: 0.5158
Epoch 6/10
189/189 [==============================] - 381s 2s/step - loss: 1.7217 - accuracy: 0.5079 - val_loss: 1.6826 - val_accuracy: 0.5250
Epoch 7/10
189/189 [==============================] - 363s 2s/step - loss: 1.5113 - accuracy: 0.5665 - val_loss: 1.5603 - val_accuracy: 0.5233

In [39]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),callbacks = [early_stopping_callback]) 

Epoch 1/10
189/189 [==============================] - 367s 2s/step - loss: 0.7286 - accuracy: 0.8009 - val_loss: 0.9887 - val_accuracy: 0.6950
Epoch 2/10
189/189 [==============================] - 382s 2s/step - loss: 0.7272 - accuracy: 0.8059 - val_loss: 1.1395 - val_accuracy: 0.6258
Epoch 3/10
189/189 [==============================] - 391s 2s/step - loss: 0.6924 - accuracy: 0.8112 - val_loss: 1.0102 - val_accuracy: 0.6833
Epoch 4/10
189/189 [==============================] - 388s 2s/step - loss: 0.6354 - accuracy: 0.8277 - val_loss: 0.9391 - val_accuracy: 0.7150
Epoch 5/10
189/189 [==============================] - 384s 2s/step - loss: 0.5649 - accuracy: 0.8585 - val_loss: 0.9025 - val_accuracy: 0.7217
Epoch 6/10
189/189 [==============================] - 369s 2s/step - loss: 0.5344 - accuracy: 0.8697 - val_loss: 0.9514 - val_accuracy: 0.7158
Epoch 7/10
189/189 [==============================] - 376s 2s/step - loss: 0.5048 - accuracy: 0.8707 - val_loss: 0.8660 - val_accuracy: 0.7258

In [40]:
model_pret_x.save('/kaggle/working/model_vgg_224_hist_1.h5')

In [ ]:
#adding more dense layers

## xcep 224 2 dense layers(relu) histogram

In [42]:
Xcep_pt_model = applications.xception.Xception(
    include_top=False,
    weights='imagenet',
)
Xcep_pt_model.trainable = False

83683744/83683744 [==============================] - 4s 0us/step


In [43]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(Xcep_pt_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(128, activation = 'relu'))

model_pret_x.add(Dense(64, activation = 'relu'))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))



In [44]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [45]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [46]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),callbacks = [early_stopping_callback]) 

Epoch 1/10
189/189 [==============================] - 423s 2s/step - loss: 2.5486 - accuracy: 0.2976 - val_loss: 2.1685 - val_accuracy: 0.3417
Epoch 2/10
189/189 [==============================] - 393s 2s/step - loss: 2.1419 - accuracy: 0.3489 - val_loss: 2.0020 - val_accuracy: 0.3800
Epoch 3/10
189/189 [==============================] - 430s 2s/step - loss: 1.9849 - accuracy: 0.4084 - val_loss: 1.9450 - val_accuracy: 0.3908
Epoch 4/10
189/189 [==============================] - 402s 2s/step - loss: 1.8996 - accuracy: 0.4187 - val_loss: 1.8781 - val_accuracy: 0.4208
Epoch 5/10
189/189 [==============================] - 397s 2s/step - loss: 1.8245 - accuracy: 0.4418 - val_loss: 1.7488 - val_accuracy: 0.4533
Epoch 6/10
189/189 [==============================] - 399s 2s/step - loss: 1.7019 - accuracy: 0.4626 - val_loss: 1.6553 - val_accuracy: 0.4783
Epoch 7/10
189/189 [==============================] - 396s 2s/step - loss: 1.6022 - accuracy: 0.5010 - val_loss: 1.6159 - val_accuracy: 0.4817

In [47]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),callbacks = [early_stopping_callback]) 

Epoch 1/10
189/189 [==============================] - 406s 2s/step - loss: 1.3166 - accuracy: 0.5635 - val_loss: 1.3818 - val_accuracy: 0.5567
Epoch 2/10
189/189 [==============================] - 395s 2s/step - loss: 1.2379 - accuracy: 0.5800 - val_loss: 1.4251 - val_accuracy: 0.5567
Epoch 3/10
189/189 [==============================] - 401s 2s/step - loss: 1.1721 - accuracy: 0.5982 - val_loss: 1.3426 - val_accuracy: 0.5917
Epoch 4/10
189/189 [==============================] - 398s 2s/step - loss: 1.0771 - accuracy: 0.6386 - val_loss: 1.3706 - val_accuracy: 0.5850
Epoch 5/10
189/189 [==============================] - 402s 2s/step - loss: 1.0263 - accuracy: 0.6478 - val_loss: 1.3573 - val_accuracy: 0.5750
Epoch 6/10
189/189 [==============================] - 393s 2s/step - loss: 0.9726 - accuracy: 0.6706 - val_loss: 1.2672 - val_accuracy: 0.6117
Epoch 7/10
189/189 [==============================] - 395s 2s/step - loss: 0.9413 - accuracy: 0.6885 - val_loss: 1.4308 - val_accuracy: 0.5517

In [48]:
model_pret_x.save('/kaggle/working/model_xcep_dense_224_hist_1.h5')

## xcep dense sigmoid 224 histo

In [13]:
Xcep_pt_model = applications.xception.Xception(
    include_top=False,
    weights='imagenet',
)
Xcep_pt_model.trainable = False

83683744/83683744 [==============================] - 3s 0us/step


In [14]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(Xcep_pt_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(128, activation = 'sigmoid'))

model_pret_x.add(Dense(64, activation = 'sigmoid'))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))



In [15]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [16]:
model_pret_xearly_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True).compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [17]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),epochs = 10, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),callbacks = [early_stopping_callback]) 

Epoch 1/10
189/189 [==============================] - 453s 2s/step - loss: 2.8468 - accuracy: 0.2662 - val_loss: 2.6512 - val_accuracy: 0.2967
Epoch 2/10
189/189 [==============================] - 395s 2s/step - loss: 2.6029 - accuracy: 0.3013 - val_loss: 2.4040 - val_accuracy: 0.2967
Epoch 3/10
189/189 [==============================] - 400s 2s/step - loss: 2.3238 - accuracy: 0.3360 - val_loss: 2.2137 - val_accuracy: 0.3542
Epoch 4/10
189/189 [==============================] - 402s 2s/step - loss: 2.2041 - accuracy: 0.3631 - val_loss: 2.1483 - val_accuracy: 0.3483
Epoch 5/10
189/189 [==============================] - 396s 2s/step - loss: 2.1264 - accuracy: 0.3879 - val_loss: 2.0886 - val_accuracy: 0.4092
Epoch 6/10
189/189 [==============================] - 405s 2s/step - loss: 2.0955 - accuracy: 0.3786 - val_loss: 2.0630 - val_accuracy: 0.3750
Epoch 7/10
189/189 [==============================] - 412s 2s/step - loss: 2.0046 - accuracy: 0.4130 - val_loss: 2.0036 - val_accuracy: 0.3950

In [18]:
model_pret_x.save('/kaggle/working/model_xcep_dense_sig_224_hist_1.h5')

## including preprocess images from the pretrained models
## 1. no normalisation

In [35]:
Xcep_pt_model = applications.xception.Xception(
    include_top=False,
    weights='imagenet',
)
Xcep_pt_model.trainable = False

In [36]:
model_pret_x = Sequential()
model_pret_x.add(TimeDistributed(Xcep_pt_model, input_shape= (20,224,224,3)))

model_pret_x.add(TimeDistributed(GlobalMaxPool2D()))

model_pret_x.add(LSTM(128))

model_pret_x.add(Dense(128, activation = 'sigmoid'))

# model_pret_x.add(Dense(64, activation = 'sigmoid'))

model_pret_x.add(Dense(len(req_classes), activation = 'softmax'))



In [37]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

In [38]:
model_pret_x.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

In [ ]:
history_pret_x = model_pret_x.fit(VideoGenerator(X = train_path_list, Y= np.array(train_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),epochs = 20, 
                            validation_data = VideoGenerator(X = test_path_list, Y= np.array(test_y_hot), batch_size = 16, 
                            frame_selection = 'hist_difference'),callbacks = [early_stopping_callback]) 

Epoch 1/20
189/189 [==============================] - 371s 2s/step - loss: 2.8245 - accuracy: 0.2811 - val_loss: 2.5148 - val_accuracy: 0.2967
Epoch 2/20
189/189 [==============================] - ETA: 0s - loss: 2.4529 - accuracy: 0.3026

In [25]:
model_pret_x.save('/kaggle/working/model_xcep_dense_sig_224_hist_new.h5')